In [1]:
!pip install pandas numpy==1.26.4 matplotlib pyarrow scikit-learn kamoe==0.1.8 xgboost  "jax[cuda12]" 

In [2]:
import os
BACKEND = 'jax' # You can use any backend here 
os.environ['KERAS_BACKEND'] = BACKEND

import time
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Flatten, GRU
from keras.optimizers import Adam

from sklearn.metrics import root_mean_squared_error
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from kamoe import MoE, KAMoE
from keras_efficient_kan import KANLinear


keras.utils.set_random_seed(1) 

N_MAX_EPOCHS = 1000
BATCH_SIZE = 128
early_stopping_callback = lambda : keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=10,
    mode="min",
    restore_best_weights=True,
    start_from_epoch=6,
)
lr_callback = lambda : keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=5,
    mode="min",
    min_delta=0.00001,
    min_lr=0.000025,
    verbose=0,
)
callbacks = lambda : [early_stopping_callback(), lr_callback(), keras.callbacks.TerminateOnNaN()]


In [3]:
# Load the dataset
california = fetch_california_housing()
X, y = california.data, california.target

# Split the data
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16512, 8) (4128, 8) (16512,) (4128,)


In [4]:
#### Define your models and configurations
models = ["MLP", "KANLinear", "XGBoost", "RandomForest", "LinearRegression", "Ridge", "Lasso", "SVR", "KNeighborsRegressor"]

nn_variants = ["Standard", "MoE", "KAMoE"]
nn_layers = [1, 2, 3, 4]
units = [5, 10, 25, 50, 100, 200, 400, 800]

# Initialize results dictionaries
results = {stat: {metric: pd.DataFrame(index=units, columns=pd.MultiIndex.from_product([models, nn_variants, nn_layers])) 
           for metric in ['r2', 'mse', 'time', 'params']} for stat in ['mean', 'std']}

n_runs = 5

def create_nn_model(model_type, variant, n_layers, n_units, input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    
    for _ in range(n_layers):
        if model_type == 'MLP':
            if variant == 'Standard':
                model.add(Dense(n_units, activation='relu'))
            elif variant == 'MoE':
                model.add(MoE(Dense(n_units, activation='relu')))
            elif variant == 'KAMoE':
                model.add(KAMoE(Dense(n_units, activation='relu')))
        elif model_type == 'KANLinear':
            if variant == 'Standard':
                model.add(KANLinear(n_units))
            elif variant == 'MoE':
                model.add(MoE(KANLinear(n_units)))
            elif variant == 'KAMoE':
                model.add(KAMoE(KANLinear(n_units)))
    
    model.add(Dense(1, activation='linear'))
    return model

for n_unit in units:
    for model_id in models:
        for variant in nn_variants:
            for n_layer in nn_layers:
                if model_id in ['LinearRegression', 'Ridge', 'Lasso', 'SVR', 'KNeighborsRegressor', 'XGBoost', 'RandomForest'] and (variant != 'Standard' or n_layer != 1):
                    continue  # Skip non-applicable configurations for non-NN models
                
                r2_scores = []
                mse_scores = []
                times = []
                n_params = []
                
                for run in range(n_runs):
                    optimizer = None
                    start_time = time.time()
                    
                    if model_id == 'LinearRegression':
                        model = LinearRegression()
                    elif model_id == 'Ridge':
                        model = Ridge(alpha=1.0, random_state=run)
                    elif model_id == 'Lasso':
                        model = Lasso(alpha=1.0, random_state=run)
                    elif model_id == 'SVR':
                        model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
                    elif model_id == 'KNeighborsRegressor':
                        model = KNeighborsRegressor(n_neighbors=5)
                    elif model_id == 'XGBoost':
                        model = XGBRegressor(n_estimators=100, learning_rate=0.01, max_depth=4, random_state=run)
                    elif model_id == 'RandomForest':
                        model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=run)
                    else:  # Neural Network models
                        model = create_nn_model(model_id, variant, n_layer, n_unit, X_train.shape[1:])
                        optimizer = Adam(0.001)
                        model.compile(optimizer=optimizer, loss='mean_squared_error')
                    
                    if isinstance(model, Sequential):
                        model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, 
                                  validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose=False)
                    else:
                        model.fit(X_train, y_train)
                    
                    end_time = time.time()
                    times.append(end_time - start_time)
                    
                    if isinstance(model, Sequential):
                        preds = model.predict(X_test, verbose=False)
                    else:
                        preds = model.predict(X_test)
                    
                    r2_scores.append(r2_score(y_test, preds))
                    mse_scores.append(mean_squared_error(y_test, preds))
                
                    if isinstance(model, Sequential):
                        n_params.append(model.count_params())
                    elif isinstance(model, XGBRegressor):
                        booster = model.get_booster()
                        tree_info = booster.get_dump(dump_format='json')
                        n_params.append(sum(len(json.loads(tree)) for tree in tree_info))
                    elif isinstance(model, RandomForestRegressor):
                        n_params.append(sum(tree.tree_.node_count for tree in model.estimators_))
                    elif isinstance(model, KNeighborsRegressor):
                        # KNN doesn't have parameters in the same sense as other models
                        n_params.append(0)
                    elif isinstance(model, SVR):
                        # For SVR, we can use the number of support vectors as a proxy for complexity
                        n_params.append(model.support_vectors_.shape[0] * model.support_vectors_.shape[1])
                    else:
                        # For linear models (LinearRegression, Ridge, Lasso)
                        n_params.append(model.coef_.size + (1 if hasattr(model, 'intercept_') else 0))

                    if isinstance(model, Sequential):
                        keras.backend.clear_session()
                    if 'model' in locals():
                        del model
                    if 'optimizer' in locals():
                        del optimizer

                # Store results
                results['std']['r2'].loc[n_unit, (model_id, variant, n_layer)] = np.std(r2_scores)
                results['std']['mse'].loc[n_unit, (model_id, variant, n_layer)] = np.std(mse_scores)
                results['std']['time'].loc[n_unit, (model_id, variant, n_layer)] = np.std(times)
                results['std']['params'].loc[n_unit, (model_id, variant, n_layer)] = np.std(n_params)
                results['mean']['r2'].loc[n_unit, (model_id, variant, n_layer)] = np.mean(r2_scores)
                results['mean']['mse'].loc[n_unit, (model_id, variant, n_layer)] = np.mean(mse_scores)
                results['mean']['time'].loc[n_unit, (model_id, variant, n_layer)] = np.mean(times)
                results['mean']['params'].loc[n_unit, (model_id, variant, n_layer)] = np.mean(n_params)
                
                print(n_unit, model_id, variant, n_layer, results['mean']['r2'].loc[n_unit, (model_id, variant, n_layer)], results['std']['r2'].loc[n_unit, (model_id, variant, n_layer)], results['mean']['time'].loc[n_unit, (model_id, variant, n_layer)], results['mean']['params'].loc[n_unit, (model_id, variant, n_layer)])


for stat in results:
    for metric in results[stat]:
        # Remove columns with all NaN values
        results[stat][metric] = results[stat][metric].dropna(axis=1, how='all')
        
        # For non-NN models, keep only the relevant columns and adjust their MultiIndex
        for model in ['XGBoost', 'RandomForest']:
            if (model, 'Standard', 1) in results[stat][metric].columns:
                results[stat][metric] = results[stat][metric].rename(columns={(model, 'Standard', 1): (model, 'Standard', 1)})
                # Remove other variants and layer numbers for non-NN models
                results[stat][metric] = results[stat][metric].drop(columns=[col for col in results[stat][metric].columns 
                                                               if col[0] == model and (col[1] != 'Standard' or col[2] != 1)])
        
        # Sort columns
        column_order = sorted(results[stat][metric].columns, key=lambda x: (x[0], x[1], x[2]))
        results[stat][metric] = results[stat][metric][column_order]

# Print results
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Don't wrap to multiple lines
for metric in ['r2', 'mse', 'time', 'params']:
    for stat in ['mean', 'std']:
        print(f"\n{metric.upper()} - {stat}:")
        display(results[stat][metric])

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

5 MLP Standard 1 0.6952889670879667 0.025889298269358353 17.58589372634888 51.0
5 MLP Standard 2 0.72791450087378 0.01353470619525801 22.59117522239685 81.0
5 MLP Standard 3 0.7263095342107784 0.007486346606130658 18.95140919685364 111.0
5 MLP Standard 4 0.7270135741136425 0.007485402672509503 14.865823221206664 141.0
5 MLP MoE 1 0.7553354502741922 0.01002993890903506 28.423695755004882 272.0
5 MLP MoE 2 0.7616899580694618 0.007901990474739995 43.67014327049255 475.0
5 MLP MoE 3 0.7672454063217522 0.007807761597730296 53.50436854362488 678.0
5 MLP MoE 4 0.7581362215281672 0.008976424353889395 67.07501826286315 881.0
5 MLP KAMoE 1 0.7527891021452672 0.012112153401557742 50.60939068794251 430.0
5 MLP KAMoE 2 0.7636938799358841 0.012634053213563265 79.67224793434143 791.0
5 MLP KAMoE 3 0.7689604414856874 0.005621342439608169 75.22971119880677 1152.0
5 MLP KAMoE 4 0.7684368593909946 0.005275557883938035 80.74803471565247 1513.0
5 KANLinear Standard 1 0.620884514060512 0.001592123153021121 

KANLinear                                                              \
        KAMoE                                     MoE                       
            1         2         3         4         1         2         3   
5    0.767117  0.761571  0.765762  0.754689  0.758368  0.768889  0.756566   
10   0.764963  0.777266  0.760414  0.752975   0.76768  0.770872  0.758268   
25   0.768027  0.769552  0.753259  0.744109  0.771841  0.766608  0.750357   
50   0.775916  0.765173  0.750816  0.744938  0.775355  0.760907  0.744733   
100  0.782804  0.765253  0.759688  0.755763  0.780125  0.761956  0.748064   
200  0.783193   0.76246  0.764725  0.769533  0.784175  0.763481  0.768033   
400  0.781442  0.747833  0.768293  0.771628  0.785393  0.746076  0.768221   
800  0.781899  0.740803  0.764839  0.772842  0.788896  0.741779  0.768535   

                                                      KNeighborsRegressor  \
               Standard                                          Standard   
            4         1         2         3         4                   1   
5    0.751131  0.620885  0.691055  0.688338  0.670526             0.67001   
10   0.750792  0.621456  0.705004  0.697473  0.682677             0.67001   
25   0.735695   0.62142   0.69883  0.694099   0.68179             0.67001   
50    0.74694  0.621627  0.699258  0.686433  0.684306             0.67001   
100  0.757984  0.621219  0.692991  0.683761  0.684893             0.67001   
200   0.76493   0.62171   0.69273  0.688903  0.697994             0.67001   
400  0.770943  0.621852  0.689021  0.698701  0.705201             0.67001   
800  0.773394  0.622034  0.695649  0.703336  0.707743             0.67001   

        Lasso LinearRegression       MLP                                \
     Standard         Standard     KAMoE                                 
            1                1         1         2         3         4   
5   -0.000219         0.575788  0.752789  0.763694   0.76896  0.768437   
10  -0.000219         0.575788  0.767538  0.774749  0.772107  0.774252   
25  -0.000219         0.575788  0.775091  0.777165  0.782224  0.782617   
50  -0.000219         0.575788  0.776886  0.786277  0.787505  0.787498   
100 -0.000219         0.575788  0.780725  0.790828  0.794667  0.791734   
200 -0.000219         0.575788  0.783449  0.792098  0.795887  0.796548   
400 -0.000219         0.575788   0.78504  0.795532  0.798928  0.797335   
800 -0.000219         0.575788  0.787932  0.797576  0.800841  0.797579   

                                                                           \
          MoE                                Standard                       
            1         2         3         4         1         2         3   
5    0.755335   0.76169  0.767245  0.758136  0.695289  0.727915   0.72631   
10   0.763681  0.768585   0.77622  0.779786  0.733158  0.754998   0.75868   
25   0.774601  0.782786  0.780922  0.783055  0.750549  0.773301  0.779673   
50   0.774212  0.784505  0.787153  0.788857  0.756356   0.78134  0.784536   
100  0.781769  0.788451  0.792335  0.792072  0.761965   0.79026  0.794054   
200  0.784175  0.791525  0.793985  0.796722   0.76349  0.794276  0.795034   
400  0.785925  0.798033  0.797955  0.794662  0.764824  0.801025  0.800077   
800  0.786353   0.79712  0.798486   0.79797  0.771809  0.802948  0.800069   

              RandomForest     Ridge       SVR   XGBoost  
                  Standard  Standard  Standard  Standard  
            4            1         1         1         1  
5    0.727014     0.774564  0.575816  0.727564  0.537515  
10   0.764335     0.774564  0.575816  0.727564  0.537515  
25   0.784873     0.774564  0.575816  0.727564  0.537515  
50   0.786329     0.774564  0.575816  0.727564  0.537515  
100  0.792927     0.774564  0.575816  0.727564  0.537515  
200  0.795569     0.774564  0.575816  0.727564  0.537515  
400  0.796746     0.774564  0.575816  0.727564  0.537515  
800  0.800456     0.774564  0.575816  0.727564  0.53751


R2 - std:


KANLinear                                                              \
        KAMoE                                     MoE                       
            1         2         3         4         1         2         3   
5    0.006075  0.008118  0.005748  0.010454  0.006773  0.003389  0.004359   
10   0.009862  0.005364  0.008945  0.005492  0.005879  0.003095  0.010625   
25    0.00386  0.004941    0.0074  0.007946  0.003322   0.00532  0.004472   
50   0.006183  0.003342    0.0085  0.011343  0.004042  0.001959  0.007053   
100   0.00426  0.002796  0.005666  0.007207  0.002986  0.002277  0.012258   
200  0.002678   0.00407  0.003769  0.006647  0.002944  0.004263  0.005603   
400  0.003281  0.003134  0.005797  0.005839  0.003056  0.006631  0.003919   
800  0.004869  0.016484  0.017984  0.008248  0.005457  0.007232  0.002937   

                                                      KNeighborsRegressor  \
               Standard                                          Standard   
            4         1         2         3         4                   1   
5    0.009537  0.001592  0.011902  0.012396  0.012388                 0.0   
10    0.00312  0.000689   0.00669  0.005015  0.009973                 0.0   
25   0.008957  0.000763  0.003467  0.002191   0.00467                 0.0   
50    0.00831  0.000734  0.004157  0.008941  0.005769                 0.0   
100  0.003634  0.000726  0.003209  0.003954  0.005826                 0.0   
200  0.004674  0.000351  0.004361  0.003982   0.00408                 0.0   
400  0.001497   0.00037  0.002273  0.004103  0.002116                 0.0   
800  0.001393   0.00011  0.001613  0.002005  0.003154                 0.0   

       Lasso LinearRegression       MLP                                \
    Standard         Standard     KAMoE                                 
           1                1         1         2         3         4   
5        0.0              0.0  0.012112  0.012634  0.005621  0.005276   
10       0.0              0.0  0.003557  0.004751   0.00233  0.005279   
25       0.0              0.0  0.006923  0.004302   0.00374  0.003289   
50       0.0              0.0  0.001971  0.003268   0.00385  0.002054   
100      0.0              0.0   0.00473  0.005564  0.002022  0.003695   
200      0.0              0.0  0.004833  0.003435  0.001874  0.002275   
400      0.0              0.0  0.001879  0.002824  0.002093   0.00218   
800      0.0              0.0  0.002275  0.003334   0.00421  0.002288   

                                                                           \
          MoE                                Standard                       
            1         2         3         4         1         2         3   
5     0.01003  0.007902  0.007808  0.008976  0.025889  0.013535  0.007486   
10   0.005024  0.007906   0.00459  0.004545  0.009599  0.005275  0.004675   
25   0.007092  0.004255   0.00518  0.003527  0.002869   0.00239  0.002448   
50   0.003253  0.002221  0.002995  0.002293  0.000909  0.002896  0.004094   
100    0.0042  0.002069  0.004269  0.002908  0.002207  0.000654  0.003632   
200  0.003735  0.004406  0.001656  0.002482  0.002284  0.003181  0.000852   
400  0.004266  0.003836  0.001682  0.003709  0.002463  0.001472  0.001888   
800  0.001548   0.00079  0.001662  0.002042  0.002168  0.001133  0.001214   

              RandomForest    Ridge      SVR  XGBoost  
                  Standard Standard Standard Standard  
            4            1        1        1        1  
5    0.007485     0.000461      0.0      0.0      0.0  
10   0.004142     0.000461      0.0      0.0      0.0  
25   0.003723     0.000461      0.0      0.0      0.0  
50   0.003927     0.000461      0.0      0.0      0.0  
100  0.001056     0.000461      0.0      0.0      0.0  
200  0.003194     0.000461      0.0      0.0      0.0  
400  0.001571     0.000461      0.0      0.0      0.0  
800  0.002454     0.000461      0.0      0.0      0.0


MSE - mean:


...

KANLinear                                                       \
         KAMoE                                        MoE             
             1          2          3         4          1         2   
5    11.097128  23.077832  19.508427  6.909309  41.942549  28.03468   
10   32.955288   9.466826    3.88361  9.787073    9.28147  6.546806   
25   13.105318   5.383108   2.295576  5.421903    5.09371  1.695311   
50   11.630839   4.179776   2.124912  3.729634   4.015101  1.611738   
100   7.720749    1.39668   2.485735  3.891512   3.205635  2.200219   
200  12.002649   2.946419   2.317763  2.905307   3.037432   1.10973   
400   8.576743    1.86131   2.583618  3.274634   4.532485  1.662063   
800    13.5761   4.395746   5.782187  7.159141   3.624675  1.537007   

                                                                    \
                           Standard                                  
             3          4         1         2         3          4   
5    22.882682  24.550971  1.593067  8.309483  2.363816  23.013194   
10    5.083446   7.608011  1.342348   0.60286  2.102463   5.781624   
25    0.728821   3.365639  1.256587  1.222201  0.395309   0.720326   
50     3.10278    2.68679  1.204835  0.713441  0.441299   0.491322   
100   4.849736   1.436204  1.332984  1.400017  0.567956   0.789446   
200   2.444065   0.704628  1.925482  1.072437  0.676019   0.702953   
400   1.828885   2.814829  1.045033  1.203022  0.471947   0.623174   
800   3.578368   1.827835   0.78271  1.624922  0.488054   0.441196   

    KNeighborsRegressor     Lasso LinearRegression        MLP             \
               Standard  Standard         Standard      KAMoE              
                      1         1                1          1          2   
5              0.000657  0.010675         0.000942  27.301404  43.141663   
10             0.000486  0.019354         0.001495   9.937386   5.494867   
25             0.000395  0.019172          0.00162  15.695496   7.141877   
50             0.000256  0.040386         0.000124   6.746081   4.517776   
100            0.000219  0.028664         0.000117   6.278563  11.945828   
200            0.000245  0.004877         0.000257   2.931771   8.985713   
400            0.000172  0.022741          0.00009   5.843163   8.804989   
800            0.000578  0.020796         0.000177  11.940093   18.28275   

                                                                      \
                                 MoE                                   
             3          4          1          2         3          4   
5     12.97042  11.579099   7.908329   12.85615  6.984075  12.856957   
10   14.045425  13.706812  16.938361   6.071045  15.29975  16.796228   
25    8.655066  16.805961  12.215263   6.524729   3.32876   3.385787   
50   10.680896   7.829397   3.198408   6.027661   7.25019   3.322059   
100   8.855039  13.531024   4.410205  10.039232    1.6882    3.15593   
200   5.638993   3.717433   3.443353   4.311426  5.815887   1.689182   
400   3.016883   2.013161   5.402185   3.776356  3.450385    2.98283   
800   6.759661   3.142345    2.81467   2.693307  3.662085   1.769755   

                                              RandomForest     Ridge  \
      Standard                                    Standard  Standard   
             1          2         3         4            1         1   
5     5.984578  10.527825  7.575085  2.270128     0.030325  0.000334   
10    4.700229   7.007934  1.118151  13.52101     0.020579  0.001142   
25   12.279803   8.654605  1.719693  1.952659     0.032459  0.009273   
50    7.723465   4.267013  2.734651  4.250934      0.01515  0.000089   
100   7.211639   3.034136  0.944897  0.489544     0.013505  0.000096   
200   6.067372   4.211119  2.219925  0.376409     0.026538  0.000094   
400   2.978742   2.909647  0.860384  0.558714     0.020226   0.00005   
800   3.708381   1.979517  0.839724  0.550007     0.072297  0.000077   

          SVR   XGBoost  
   


PARAMS - mean:


KANLinear                                                            \
        KAMoE                                           MoE               
            1           2           3           4         1           2   
5      1228.0      2099.0      2970.0      3841.0    1070.0      1783.0   
10     2253.0      4949.0      7645.0     10341.0    2005.0      4453.0   
25     5328.0     20099.0     34870.0     49641.0    4810.0     19063.0   
50    10453.0     67349.0    124245.0    181141.0    9485.0     65413.0   
100   20703.0    244349.0    467995.0    691641.0   18835.0    240613.0   
200   41203.0    928349.0   1815495.0   2702641.0   37535.0    921013.0   
400   82203.0   3616349.0   7150495.0  10684641.0   74935.0   3601813.0   
800  164203.0  14272349.0  28380495.0  42488641.0  149735.0  14243413.0   

                                                                        \
                            Standard                                     
              3           4        1          2          3           4   
5        2496.0      3209.0    317.0      517.0      717.0       917.0   
10       6901.0      9349.0    607.0     1347.0     2087.0      2827.0   
25      33316.0     47569.0   1477.0     5937.0    10397.0     14857.0   
50     121341.0    177269.0   2927.0    20587.0    38247.0     55907.0   
100    462391.0    684169.0   5827.0    76137.0   146447.0    216757.0   
200   1804491.0   2687969.0  11627.0   292237.0   572847.0    853457.0   
400   7128691.0  10655569.0  23227.0  1144437.0  2265647.0   3386857.0   
800  28337091.0  42430769.0  46427.0  4528837.0  9011247.0  13493657.0   

    KNeighborsRegressor    Lasso LinearRegression      MLP             \
               Standard Standard         Standard    KAMoE              
                      1        1                1        1          2   
5                   0.0      9.0              9.0    430.0      791.0   
10                  0.0      9.0              9.0    735.0     1541.0   
25                  0.0      9.0              9.0   1650.0     4991.0   
50                  0.0      9.0              9.0   3175.0    14741.0   
100                 0.0      9.0              9.0   6225.0    49241.0   
200                 0.0      9.0              9.0  12325.0   178241.0   
400                 0.0      9.0              9.0  24525.0   676241.0   
800                 0.0      9.0              9.0  48925.0  2632241.0   

                                                                              \
                               MoE                                  Standard   
             3          4        1          2          3          4        1   
5       1152.0     1513.0    272.0      475.0      678.0      881.0     51.0   
10      2347.0     3153.0    487.0     1045.0     1603.0     2161.0    101.0   
25      8332.0    11673.0   1132.0     3955.0     6778.0     9601.0    251.0   
50     26307.0    37873.0   2207.0    12805.0    23403.0    34001.0    501.0   
100    92257.0   135273.0   4357.0    45505.0    86653.0   127801.0   1001.0   
200   344157.0   510073.0   8657.0   170905.0   333153.0   495401.0   2001.0   
400  1327957.0  1979673.0  17257.0   661705.0  1306153.0  1950601.0   4001.0   
800  5215557.0  7798873.0  34457.0  2603305.0  5172153.0  7741001.0   8001.0   

                                    RandomForest    Ridge       SVR  XGBoost  
                                        Standard Standard  Standard Standard  
            2          3          4            1        1         1        1  
5        81.0      111.0      141.0     140657.6      9.0  102512.0    800.0  
10      211.0      321.0      431.0     140657.6      9.0  102512.0    800.0  
25      901.0     1551.0     2201.0     140657.6      9.0  102512.0    800.0  
50     3051.0     5601.0     8151.0     140657.6      9.0  102512.0    800.0  
100   11101.0    21201.0    31301.0     140657.6      9.0  102512.0    800.0  
200   42201.0    82401.0   122601.0     140657


PARAMS - std:


KANLinear                                                             \
        KAMoE                 MoE                Standard                  
            1    2    3    4    1    2    3    4        1    2    3    4   
5         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
10        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
25        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
50        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
100       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
200       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
400       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   
800       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      0.0  0.0  0.0  0.0   

    KNeighborsRegressor    Lasso LinearRegression   MLP                      \
               Standard Standard         Standard KAMoE                 MoE   
                      1        1                1     1    2    3    4    1   
5                   0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
10                  0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
25                  0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
50                  0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
100                 0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
200                 0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
400                 0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   
800                 0.0      0.0              0.0   0.0  0.0  0.0  0.0  0.0   

                                           RandomForest    Ridge      SVR  \
                   Standard                    Standard Standard Standard   
       2    3    4        1    2    3    4            1        1        1   
5    0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
10   0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
25   0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
50   0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
100  0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
200  0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
400  0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   
800  0.0  0.0  0.0      0.0  0.0  0.0  0.0   675.023437      0.0      0.0   

     XGBoost  
    Standard  
           1  
5        0.0  
10       0.0  
25       0.0  
50       0.0  
100      0.0  
200      0.0  
400      0.0  
800      0.0